# Unit conversion notes

## Follow-up work

 - Change name to Display Units
 - Which plugins will break
   - Cubeviz slider
   - Model fitting
   - Line analysis
 - Floating point conversion error?
 - Mouse events account for display units
 - Default to first data loaded for display units
 - Global units which plugins can listen to. Maybe a display_unit_changed message?
 - Should get data listen to display units?
 - Feature branch or develop in place?
---

## Acceptance criteria

 - Equivalencies list in notebook
 - Make tickets
 ---
 
## Examples 
 Can make multiple unit_converter instances, which can handle different cases in Jdaviz
 
 - from glue.config import settings
 - settings.UNIT_CONVERTER = 'custom-astropy'
 - @unit_converter('custom-jdaviz')
 - class UnitConverterWithSpectral:

Using d1 below:
```
wcs1 = WCS(naxis=1)
wcs1.wcs.ctype = ['FREQ']
wcs1.wcs.crval = [1]
wcs1.wcs.cdelt = [1]
wcs1.wcs.crpix = [1]
wcs1.wcs.cunit = ['GHz']

d1 = Data(f1=[1, 2, 3])
d1.get_component('f1').units = 'Jy'
d1.coords = wcs1
```
access with
```
astropy_converter = unit_converter.members[settings.UNIT_CONVERTER]()
equivalent_units = astropy_converter.equivalent_units(d1, d1.components[0], 'mJy')
converted_units = astropy_converter.to_unit(d1, 'Pixel Axis 0 [x]', d1.x, 'GHz', 'centimeter')
```
---
Dev notes

y display unit is None and x display unit is set to 'Hz'
y axis label is Data values
x label is Frequency \[Hz\]

In [ ]:
import warnings

import numpy as np
from astropy.wcs import WCS
import glue_jupyter as gj
from glue.core import Data
from glue.plugins.wcs_autolinking.wcs_autolinking import WCSLink
from specutils import Spectrum1D
from glue.core.link_helpers import LinkSame
from glue.core.roi import XRangeROI
from glue.config import settings


from jdaviz import Specviz

In [ ]:
from astropy import units as u
from glue.core.units import unit_converter


@unit_converter('custom-jdaviz')
class UnitConverterWithSpectral:

    @staticmethod
    def _cid_is_flux(cid):
        return cid in ('f1', 'f2')
    
    def equivalent_units(self, data, cid, units):
        # Given a glue data object (data), a component ID (cid), and the units
        # of that component in the data object (units), this method should
        # return a flat list of units (as strings) that the data could be
        # converted to. This is used to construct the drop-down menus with the
        # available units to convert to.
        if self._cid_is_flux(cid):
            available_units = ['Jy', 'mJy', 'uJy',
                               'W / (m2 Hz)',
                               'eV / (s m2 Hz)',
                               'erg / (s cm2)',
                               'erg / (s cm2 um)',
                               'erg / (s cm2 Angstrom)',
                               'erg / (s cm2 Hz)',
                               'ph / (s cm2 um)',
                               'ph / (s cm2 Angstrom)',
                               'ph / (s cm2 Hz)']
        else:  # spectral axis
            available_units = ['angstrom', 'nanometer', 'micron', 'hertz', 'erg']
        return available_units

    def to_unit(self, data, cid, values, original_units, target_units):
        # Given a glue data object (data), a component ID (cid), the values
        # to convert, and the original and target units of the values, this method
        # should return the converted values. Note that original_units
        # gives the units of the values array, which might not be the same
        # as the original native units of the component in the data.
        if self._cid_is_flux(cid):
            spec = data.get_object(cls=Spectrum1D)
            eqv = u.spectral_density(spec.spectral_axis)
        else:  # spectral axis
            eqv = u.spectral()
        return (values * u.Unit(original_units)).to_value(u.Unit(target_units), equivalencies=eqv)


settings.UNIT_CONVERTER = 'custom-jdaviz'

# The following cell changes the display of the flux unit

In [ ]:
wcs1 = WCS(naxis=1)
wcs1.wcs.ctype = ['FREQ']
wcs1.wcs.crval = [1]
wcs1.wcs.cdelt = [1]
wcs1.wcs.crpix = [1]
wcs1.wcs.cunit = ['GHz']

d1 = Data(f1=[1, 2, 3])
d1.get_component('f1').units = 'Jy'
d1.coords = wcs1

# So we can see both data around the same spectral axis space.
wcs2 = WCS(naxis=1)
wcs2.wcs.ctype = ['WAVE']
wcs2.wcs.crval = [30]
wcs2.wcs.cdelt = [-15]
wcs2.wcs.crpix = [1]
wcs2.wcs.cunit = ['cm']

d2 = Data(f2=[2000, 1000, 3000])
d2.get_component('f2').units = 'mJy'
d2.coords = wcs2

specviz = Specviz()

In [ ]:
specviz.app.add_data(d1, "d1")
specviz.app.add_data(d2, "d2")
specviz.app.data_collection.add_link(WCSLink(d1, d2))

specviz.show()

# Take a moment here to add data manually from the data drop down into the viewer.

In [ ]:
def convert_x_axis_to(new_x):
    specviz.app.get_viewer("spectrum-viewer").state.x_display_unit = new_x

def convert_y_axis_to(new_y):
    specviz.app.get_viewer("spectrum-viewer").state.y_display_unit = new_y

In [ ]:
convert_y_axis_to("Jy")

In [ ]:
# Errors out because of an issue with equivalencies

convert_x_axis_to("cm")

In [ ]:
astropy_converter = unit_converter.members[settings.UNIT_CONVERTER]()

data = specviz.app.data_collection[0]
spec = data.get_object(cls=Spectrum1D)
print(list(astropy_converter.equivalent_units(data, 'Pixel Axis 0 [x]', 'GHz')))
print(astropy_converter.to_unit(data, 'Pixel Axis 0 [x]', spec.spectral_axis, spec.spectral_axis.unit, 'centimeter'))

In [ ]:
# Errors out because of an issue with equivalencies

print(list(astropy_converter.equivalent_units(data, 'f1', 'Jy')))
print(astropy_converter.to_unit(data, 'f1', spec.flux, spec.flux.unit, 'Jy'))

In [ ]:
for x in specviz.app.data_collection:
    print(x.components)

In [ ]:
specviz.app.get_viewer("spectrum-viewer").apply_roi(XRangeROI(1.2e+8, 2e+9))